In [ ]:
from __future__ import print_function, division, absolute_import

import GPy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets # Widget definitions
from IPython import display # Used to display widgets in the notebook

# Import safeopt from system, alternatively add main folder to path
try:
    import safeopt
except ImportError:
    import sys
    import os
    module_path = os.path.abspath('..')
    sys.path.append(module_path)
    import safeopt

## Define a kernel and function

Here we define a kernel. The function is drawn at random from the GP and is corrupted my Gaussian noise

In [ ]:
# Measurement noise
noise_var = 0.05 ** 2

# Set fixed Gaussian measurement noise
likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)
likelihood2 = likelihood.copy()
likelihood2.variance = 1e-5

# Bounds on the inputs variable
bounds = [(-10., 10.)]

# Define Kernel
kernel = GPy.kern.RBF(input_dim=len(bounds), variance=2., lengthscale=1.0, ARD=True)
kernel2 = kernel.copy()

# set of parameters
parameter_set = safeopt.linearly_spaced_combinations(bounds, 1000)

# Initial safe point
x0 = np.zeros((1, len(bounds)))

# Generate function with safe initial point at x=0
def sample_safe_fun():
    fun = safeopt.sample_gp_function(kernel, bounds, likelihood.variance, 100)
    while True:
        fun2 = safeopt.sample_gp_function(kernel2, bounds, likelihood2.variance, 100)
        if fun2(0, noise=False) > 1:
            break
            
    def combined_fun(x, noise=True):
        return np.hstack([fun(x, noise), fun2(x, noise)])
    return combined_fun

## Interactive run of the algorithm

In [ ]:
# whether to use the swarm optimization
swarm_optimization = False

In [ ]:
button_add = widgets.Button(description='Add sample', tooltip='Sample a new data point for the optimization')
button_reset = widgets.Button(description='Reset', tooltip='Restart the safe optimization algorithm')
button_new = widgets.Button(description='New function', tooltip='Add a new function and reset')
slider_safety = widgets.FloatSlider(description='Safety threshold', value=0, min=-4, max=4, step=0.1)

ms = 20
mew = 7
lw = 5

def plot_gp():
    figure, axes = plt.subplots(2, sharex=True, figsize=(15, 2 * 8))
    
    for i in range(len(gp_opt.gps)):
        gp = gp_opt.gps[i]
        axis = axes[i]
        
        safeopt.plot_2d_gp(gp, np.linspace(bounds[0][0], bounds[0][1], 1000)[:, None], axis=axis, lw=lw)
    
        # Plot last point red
        axis.plot(gp.X[-1, :], gp.Y[-1, :],
                  'rx', ms=ms, mew=mew, label='Last Point')
        
        # Plot safe line
        axis.plot([bounds[0][0], bounds[0][1]], [gp_opt.fmin[i], gp_opt.fmin[i]],
                  'k--', label='Minimum', lw=lw)
        
        axis.plot(true_values[0], true_values[1][:, [i]],
                 alpha=0.15, label='True function', lw=lw)
    
    plt.ylim([-4, 4])
    plt.legend(loc=3)
    
    # Ensure we only get one plot
    display.clear_output(wait=True)

def new_sample(b=None):
    """Draw a new gp sample"""
    x = gp_opt.optimize()
    gp_opt.add_new_data_point(x, fun(x))
    plot_gp()
button_add.on_click(new_sample)

def reset(b=None):
    """Reset the GP-UCB algorithm"""
    global gp_opt
    y0 = fun(x0)
    gp = GPy.core.GP(x0, y0[:, 0, None], kernel, likelihood)
    gp2 = GPy.core.GP(x0, y0[:, 1, None], kernel2, likelihood2)
    if swarm_optimization:
        gp_opt = safeopt.SafeOptSwarm([gp, gp2], [-np.inf, 0.], bounds=bounds, threshold=0.2)
    else:
        gp_opt = safeopt.SafeOpt([gp, gp2], parameter_set, [-np.inf, 0.], lipschitz=None, threshold=0.2)
    plot_gp()
button_reset.on_click(reset)

def new_fun(b=None):
    """Draw a new function from the GP"""
    global fun
    global true_values
    fun = sample_safe_fun()
    inp = np.linspace(bounds[0][0], bounds[0][1], 200)[:, None]
    true_values = (inp, fun(inp, noise=False))
    reset(b)
button_new.on_click(new_fun)

def change_safety(b=None):
    if b == 'value':
        gp_opt.fmin[1] = slider_safety.get_state()['value']
        plot_gp()
slider_safety.on_trait_change(change_safety)

display.display(button_add, button_new, button_reset, slider_safety)
new_fun()